<div align=center><img src="./assets/cover.png" alt="bert-cover"></div>

<div align=center><img src="./assets/contents_1_2.png" alt="table-of-contents"></div>

<div align=center><img src="./assets/qr_code.jpeg" alt="table-of-contents" width="1300"></div>

https://github.com/mindspore-courses/step_into_chatgpt

https://github.com/mindspore-lab/mindnlp

## 3202年了，为什么还要学BERT

- ### Decoder only模型当道： GPT3、Bloom、LLAMA、GLM

    - #### Transformer Encoder结构在生成式任务上的缺陷

    - #### BERT模型规模小

    - #### Pretrain-Fintune范式的落寞

- ### 2022年以前，学术界还是在倒腾BERT

    - #### Finetune更容易针对单领域任务训练
    
    - #### BERT是首个大规模并行预训练的模型，也是当前的performance baseline
    
    - #### 由BERT入手学大模型训练、微调、Prompt最简单

# NLP中的预训练模型

语言模型的演变经历了以下几个阶段：

<div align=center><img src="./assets/language_models.svg" alt="language-models" width="1000"></div>

1. `word2vec`/`Glove`将离散的文本数据转换为固定长度的**静态词向量**，后根据下游任务训练不同的**语言模型**；

2. `ELMo`预训练模型将文本数据*结合上下文信息*，转换为**动态词向量**，后根据下游任务训练不同的**语言模型**；

3. `BERT`同样将文本数据转换为**动态词向量**，能够更好地捕捉*句子级别的信息与语境信息*，后续只需finetune最后的**输出层**即可适配下游任务；

4. `GPT`等预训练语言模型主要用于*文本生成类任务*，需要通过**prompt方法**来应用于下游任务，指导模型生成特定的输出。

# BERT

<div align=center><img src="./assets/bert-transfer-learning.png"></div>

BERT模型本质上是结合了`ELMo`模型与`GPT`模型的优势。

- 相比于ELMo，BERT仅需改动最后的输出层，而非模型架构，便可以在下游任务中达到很好的效果；
- 相比于GPT，BERT在处理词元表示时考虑到了双向上下文的信息；

BERT通过两种无监督任务（Masked Language Modelling 和 Next Sentence Prediction）进行预训练，其次，在下游任务中对预训练Transformer编码器的所有参数进行微调，额外的输出层将从头开始训练。

> Reference: [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](http://jalammar.github.io/illustrated-bert/)

# BERT 结构

BERT（Bidirectional Encoder Representation from Transformers）是由Transformer的Encoder层堆叠而成，BERT的模型大小有如下两种：

- BERT BASE：与Transformer参数量齐平，用于比较模型效果（110M parameters）
- BERT LARGE：在BERT BASE基础上扩大参数量，达到了当时各任务最好的结果（340M parameters）

| model | blocks | hidden size | attention heads |
| :-----:| :----: | :----: | :----: |
| Transformer | 6 | 512 | 8 |
| BERT BASE | 12 | 768 | 12 |
| BERT LARGE | 24 | 1024 | 16 |

> Reference: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding

<div align=center><img src="./assets/bert-base-bert-large-encoders.png"></div>

接受输入序列后，BERT会输出每个位置对应的向量（长度等于hidden size），在后续下游任务中，我们会选取与任务相关的位置的向量，输入到最终输出层中得到结果。

如在诈骗邮件分类任务中，我们会将表示句子级别信息的`[CLS]` token所对应的vector，放入classfier中，得到对spam/not spam分类的预测。

<div align=center><img src="./assets/bert-classifier.png"></div>

## BERT 输入

- 针对句子对相关任务，将两个句子合并为一个句子对输入到Encoder中，`[CLS]` + 第一个句子 + `[SEP]` + 第二个句子 + `[SEP]`;
- 针对单个文本相关任务，`[CLS]` + 句子 + 
`[SEP]`。

在诈骗邮件分类中，输入为单个句子，在拆分为tokens后，在序列首尾分别添加`[CLS]`与`[SEP]`即可。

<div align=center><img src="./assets/bert-input.png"></div>

In [1]:
# install mindnlp
!pip install mindnlp==0.4.0
!pip install pytesseract

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple/

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple/

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from mindnlp.transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sequence = 'help prince mayuko transfer huge inheritance'
model_inputs = tokenizer(sequence)
print(model_inputs)
tokens = []
for index in model_inputs['input_ids']:
    tokens.append(tokenizer.convert_ids_to_tokens(index))
print(tokens)

[WARNING] GE_ADPT(6108,ffffab724020,python):2025-05-13-12:28:08.673.160 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleGetModelId failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleGetModelId
[WARNING] GE_ADPT(6108,ffffab724020,python):2025-05-13-12:28:08.673.218 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleLoadFromMem failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleLoadFromMem
[WARNING] GE_ADPT(6108,ffffab724020,python):2025-05-13-12:28:08.673.244 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleUnload failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleUnload
[WARNING] GE_ADPT(6108,ffffab724020,python):2025-05-13-12:28:08.673.383 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymA

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

  0%|          | 0.00/334 [00:00<?, ?B/s]

{'input_ids': [101, 2393, 3159, 2089, 6968, 2080, 4651, 4121, 12839, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'help', 'prince', 'may', '##uk', '##o', 'transfer', 'huge', 'inheritance', '[SEP]']


/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


## BERT Embedding

输入到BERT模型的信息由三部分内容组成：

- 表示内容的token ids
- 表示位置的position ids
- 用于区分不同句子的token type ids

三种信息分别进入Embedding层，得到token embeddings、position embeddings与segment embeddings；与Transformer不同，以上三种均为**可学习**的信息。

<div align=center><img src="./assets/bert-embedding-modified.png" alt="bert-embedding" width="1000"></div>

In [3]:
import mindspore
from mindspore import nn
import mindspore.common.dtype as mstype
from mindspore.common.initializer import initializer, TruncatedNormal

class BertEmbeddings(nn.Cell):
    """
    Embeddings for BERT, include word, position and token_type
    """
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, embedding_table=TruncatedNormal(config.initializer_range))
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size, embedding_table=TruncatedNormal(config.initializer_range))
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size, embedding_table=TruncatedNormal(config.initializer_range))
        self.layer_norm = nn.LayerNorm((config.hidden_size,), epsilon=config.layer_norm_eps)
        self.dropout = nn.Dropout(1 - config.hidden_dropout_prob)

    def construct(self, input_ids, token_type_ids=None, position_ids=None):
        seq_len = input_ids.shape[1]
        if position_ids is None:
            position_ids = mnp.arange(seq_len)
            position_ids = position_ids.expand_dims(0).expand_as(input_ids)
        if token_type_ids is None:
            token_type_ids = ops.zeros_like(input_ids)
        
        words_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        embeddings = words_embeddings + position_embeddings + token_type_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings


## BERT 模型构建

BERT模型的构建与上一节课程的Transformer Encoder构建类似。

分别构建multi-head attention层，feed-forward network，并在中间用add&norm连接，最后通过线性层与softmax层进行输出。

<div align=center><img src="./assets/bert-model-architecture.png"></div>

### BERT self-attention 层

<div align=center><img src="./assets/transformer_multi-headed_self-attention-recap.png"></div>

In [4]:
class BertSelfAttention(nn.Cell):
    """
    Self attention layer for BERT.
    """
    def __init__(self,  config):
        super().__init__()
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                f"The hidden size {config.hidden_size} is not a multiple of the number of attention "
                f"heads {config.num_attention_heads}"
            )
        self.output_attentions = config.output_attentions

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Dense(config.hidden_size, self.all_head_size, \
            weight_init=TruncatedNormal(config.initializer_range))
        self.key = nn.Dense(config.hidden_size, self.all_head_size, \
            weight_init=TruncatedNormal(config.initializer_range))
        self.value = nn.Dense(config.hidden_size, self.all_head_size, \
            weight_init=TruncatedNormal(config.initializer_range))

        self.dropout = Dropout(config.attention_probs_dropout_prob)
        self.softmax = nn.Softmax(-1)
        self.matmul = Matmul()

    def transpose_for_scores(self, input_x):
        """
        transpose for scores
        [batch_size, seq_len, num_heads, head_size] to [batch_size, num_heads, seq_len, head_size]
        """
        new_x_shape = input_x.shape[:-1] + (self.num_attention_heads, self.attention_head_size)
        input_x = input_x.view(*new_x_shape)
        return input_x.transpose(0, 2, 1, 3)

    def construct(self, hidden_states, attention_mask=None, head_mask=None):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)
        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        attention_scores = self.matmul(query_layer, key_layer.swapaxes(-1, -2))
        attention_scores = attention_scores / ops.sqrt(Tensor(self.attention_head_size, mstype.float32))
        if attention_mask is not None:
            attention_scores = attention_scores + attention_mask

        attention_probs = self.softmax(attention_scores)

        attention_probs = self.dropout(attention_probs)

        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = self.matmul(attention_probs, value_layer)
        context_layer = context_layer.transpose(0, 2, 1, 3)
        new_context_layer_shape = context_layer.shape[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer, attention_probs) if self.output_attentions else (context_layer,)
        return outputs

### BERT self-attention 输出层 

- BERTSelfOutput：residual connection + layer normalization
- BERTAttention: self-attention + add&norm

<div align=center><img src="./assets/bert-attention-code.png"></div>

In [5]:
class BertSelfOutput(nn.Cell):
    r"""
    Bert Self Output
    self-attention output + residual connection + layer norm
    """
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Dense(config.hidden_size, config.hidden_size, \
            weight_init=TruncatedNormal(config.initializer_range))
        self.layer_norm = nn.LayerNorm((config.hidden_size,), epsilon=1e-12)
        self.dropout = Dropout(config.hidden_dropout_prob)

    def construct(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.layer_norm(hidden_states + input_tensor)
        return hidden_states

### BERT feed-forward 层

In [6]:
class BertIntermediate(nn.Cell):
    r"""
    Bert Intermediate
    """
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Dense(config.hidden_size, config.intermediate_size, \
            weight_init=TruncatedNormal(config.initializer_range))
        self.intermediate_act_fn = ACT2FN[config.hidden_act]

    def construct(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states

### BERT 最后的Add&Norm

In [7]:
class BertOutput(nn.Cell):
    r"""
    Bert Output
    """
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Dense(config.intermediate_size, config.hidden_size, \
            weight_init=TruncatedNormal(config.initializer_range))
        self.layer_norm = nn.LayerNorm((config.hidden_size,), epsilon=1e-12)
        self.dropout = Dropout(config.hidden_dropout_prob)

    def construct(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self. layer_norm(hidden_states + input_tensor)
        return hidden_states

### BERT Encoder

- BertLayer：Encoder Layer，集合了self-attention, feed-forward并通过add&norm连接
- BertEnocoder：通过Encoder Layer堆叠起来的Encoder结构

<div align=center><img src="./assets/bert-model-code.png"></div>

In [8]:
class BertLayer(nn.Cell):
    r"""
    Bert Layer
    """
    def __init__(self, config):
        super().__init__()
        self.attention = BertAttention(config)
        self.intermediate = BertIntermediate(config)
        self.output = BertOutput(config)

    def construct(self, hidden_states, attention_mask=None, head_mask=None):
        attention_outputs = self.attention(hidden_states, attention_mask, head_mask)
        attention_output = attention_outputs[0]
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        outputs = (layer_output,) + attention_outputs[1:]
        return outputs


class BertEncoder(nn.Cell):
    r"""
    Bert Encoder
    """
    def __init__(self, config):
        super().__init__()
        self.output_attentions = config.output_attentions
        self.output_hidden_states = config.output_hidden_states
        self.layer = nn.CellList([BertLayer(config) for _ in range(config.num_hidden_layers)])

    def construct(self, hidden_states, attention_mask=None, head_mask=None):
        all_hidden_states = ()
        all_attentions = ()
        for i, layer_module in enumerate(self.layer):
            if self.output_hidden_states:
                all_hidden_states += (hidden_states,)

            layer_outputs = layer_module(hidden_states, attention_mask, head_mask[i])
            hidden_states = layer_outputs[0]

            if self.output_attentions:
                all_attentions += (layer_outputs[1],)

        if self.output_hidden_states:
            all_hidden_states += (hidden_states,)

        outputs = (hidden_states,)
        if self.output_hidden_states:
            outputs += (all_hidden_states,)
        if self.output_attentions:
            outputs += (all_attentions,)
        return outputs


## BERT 输出

BERT会针对每一个位置输出大小为hidden size的向量，在下游任务中，会根据任务内容的不同，选取不同的向量放入输出层。
- 我们一般称`[CLS]`经过线性层+激活函数tanh的输出为**pooler output**，用于句子级别的分类/回归任务;
- 我们一般称BERT输出的每个位置对应的vector为**sequence output**,用于词语级别的分类任务；

<div align=center><img src="./assets/bert-output.png"></div>

### BERT Pooler

In [9]:
class BertPooler(nn.Cell):
    r"""
    Bert Pooler
    """
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Dense(config.hidden_size, config.hidden_size, \
            activation='tanh', weight_init=TruncatedNormal(config.initializer_range))

    def construct(self, hidden_states):

        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        return pooled_output

## BERT 预训练

BERT通过Masked LM（masked language model）与NSP（next sentence prediction）获取词语和句子级别的特征。

<div align=center><img src="./assets/bert_pretrain_finetune.jpg" alt="bert-pretrain" width="1000"></div>

> <font size=2>图片来源：Devlin, J.; Chang, M. W.; Lee, K.; Toutanova, K. BERT: Pre-training of deep bidirectional transformers for language understanding. arXiv preprint arXiv:1810.04805, 2018.</font>

### Masked Language Model (Masked LM)

BERT模型通过Masked LM捕捉**词语层面**的信息。

我们随机将每个句子中15%的词语进行遮盖，替换成掩码\<mask\>。在训练过程中，模型会对句子进行“完形填空”，预测这些被遮盖的词语是什么，通过减小被mask词语的损失值来对模型进行优化。

<div align=center><img src="./assets/masked_lm.png" alt="masked-lm" width="1000"></div>

> <font size=2>图片来源: [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](https://jalammar.github.io/illustrated-bert)</font>

由于\<mask\>仅在预训练中出现，为了让预训练和微调中的数据处理尽可能接近，我们在随机mask的时候进行如下操作：
- 80%的概率替换为\<mask\>
- 10%的概率替换为文本中的随机词
- 10%的概率不进行替换，保持原有的词元

<div align=center><img src="./assets/masked_lm_2.png" alt="masked-lm-2" width="1000"></div>

我们通过`BERTPredictionHeadTranform`实现单层感知机，对被遮盖的词元进行预测。在前向网络中，我们需要输入BERT模型的编码结果`hidden_states`。

In [10]:
activation_map = {
    'relu': nn.ReLU(),
    'gelu': nn.GELU(False),
    'gelu_approximate': nn.GELU(),
    'swish':nn.SiLU()
}

class BertPredictionHeadTransform(nn.Cell):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Dense(config.hidden_size, config.hidden_size, weight_init=TruncatedNormal(config.initializer_range))
        self.transform_act_fn = activation_map.get(config.hidden_act, nn.GELU(False))
        self.layer_norm = nn.LayerNorm((config.hidden_size,), epsilon=config.layer_norm_eps)
    
    def construct(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.transform_act_fn(hidden_states)
        hidden_states = self.layer_norm(hidden_states)
        return hidden_states

[WARNING] DEVICE(6108,ffffab724020,python):2025-05-13-12:28:58.320.316 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_memory_adapter.cc:116] Initialize] Free memory size is less than half of total memory size.Device 0 Device HBM total size:34359738368 Device HBM free size:66969600 may be other processes occupying this card, check as: ps -ef|grep python


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


根据被遮盖的词元位置`masked_lm_positions`，获得这些词元的预测输出。

In [11]:
import mindspore.ops as ops
import mindspore.numpy as mnp
from mindspore import Parameter, Tensor

class BertLMPredictionHead(nn.Cell):
    def __init__(self, config):
        super(BertLMPredictionHead, self).__init__()
        self.transform = BertPredictionHeadTransform(config)

        self.decoder = nn.Dense(config.hidden_size, config.vocab_size, has_bias=False, weight_init=TruncatedNormal(config.initializer_range))

        self.bias = Parameter(initializer('zeros', config.vocab_size), 'bias')

    def construct(self, hidden_states, masked_lm_positions):

        batch_size, seq_len, hidden_size = hidden_states.shape
        if masked_lm_positions is not None:
            flat_offsets = mnp.arange(batch_size) * seq_len
            flat_position = (masked_lm_positions + flat_offsets.reshape(-1, 1)).reshape(-1)
            flat_sequence_tensor = hidden_states.reshape(-1, hidden_size)
            hidden_states = ops.gather(flat_sequence_tensor, flat_position, 0)
        hidden_states = self.transform(hidden_states)
        hidden_states = self.decoder(hidden_states) + self.bias
        return hidden_states

### Next Sentence Prediction (NSP)

BERT通过NSP捕捉**句子级别**的信息，使其可以理解句子与句子之间的联系，从而能够应用于问答或者推理任务。

NSP本质上是一个**二分类任务**，通过输入一个句子对，判断两句话是否为连续句子。输入的两个句子A和B中，B有50%的概率是A的下一句。

<div align=center><img src="./assets/nsp.png" alt="nsp" width="500"></div>

> <font size=2>图片来源: [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](https://jalammar.github.io/illustrated-bert)</font>

另外，输入的内容最好是document-level的语料，而非sentence-level的语料，这样训练出的模型可以具备抓取长序列特征的能力。

在这里，我们使用一个单隐藏层的多层感知机`BERTPooler`进行二分类预测。因为特殊占位符\<cls\>在预训练中对应了句子级别的特征信息，所以多层感知机分类器只需要输出\<cls\>对应的隐藏层输出。

In [12]:
class BertPooler(nn.Cell):
    def __init__(self, config):
        super(BertPooler, self).__init__()
        self.dense = nn.Dense(config.hidden_size, config.hidden_size, activation='tanh', weight_init=TruncatedNormal(config.initializer_range))
    
    def construct(self, hidden_states):
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        return pooled_output

最后，多层感知机分类器的输出通过一个线性层`self.seq_relationship`，输出对nsp的预测。

在`BERTPreTrainingHeads`中，我们对以上提到的两种方式进行整合。最终输出Maked LM（`prediction scores`）和NSP（`seq_realtionship_score`）的预测结果。

In [13]:
class BertPreTrainingHeads(nn.Cell):
    def __init__(self, config):
        super(BertPreTrainingHeads, self).__init__()
        self.predictions = BertLMPredictionHead(config)
        self.seq_relationship = nn.Dense(config.hidden_size, 2, weight_init=TruncatedNormal(config.initializer_range))
    
    def construct(self, sequence_output, pooled_output, masked_lm_positions):
        prediction_scores = self.predictions(sequence_output, masked_lm_positions)
        seq_relationship_score = self.seq_relationship(pooled_output)
        return prediction_scores, seq_relationship_score

### BERT预训练代码整合

我们将上述的类进行实例化，并借此回顾一下BERT预训练的整体流程。

1. `BertModel`构建BERT模型；
2. `BertPretrainingHeads`整合了Masked LM与NSP两个训练任务， 输出预测结果；
    - `BertLMPredictionHead`：输入BERT编码与\<mask\>的位置，输出对应位置词元的预测；
    - `BERTPooler`：输入BERT编码，输出对\<cls\>的隐藏状态，并在`BertPretrainingHeads`中通过线性层输出预测结果；

In [14]:
class BertForPretraining(nn.Cell):
    def __init__(self, config, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.bert = BertModel(config)
        self.cls = BertPreTrainingHeads(config)
        self.vocab_size = config.vocab_size

        self.cls.predictions.decoder.weight = self.bert.embeddings.word_embeddings.embedding_table

    def construct(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, masked_lm_positions=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask
        )

        sequence_output, pooled_output = outputs[:2]
        prediction_scores, seq_relationship_score = self.cls(sequence_output, pooled_output, masked_lm_positions)

        outputs = (prediction_scores, seq_relationship_score,) + outputs[2:]

        return outputs